In [0]:
#Adam optimizer, batch size of 64, Bi Directional  GRU, first try with deep learning 

In [0]:
!pip install -q keras
import numpy as np
np.random.seed(42)
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

from keras.models import Model
from keras.layers import Input, Dense, Embedding, SpatialDropout1D, concatenate
from keras.layers import GRU, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.preprocessing import text, sequence
from keras.callbacks import Callback


In [0]:
#Upload files:

# from google.colab import files

# uploaded = files.upload()

# for fn in uploaded.keys():
#   print('User uploaded file "{name}" with length {length} bytes'.format(
#       name=fn, length=len(uploaded[fn])))

In [0]:
#using google drive:
#https://drive.google.com/open?id=1csHM7IpU3V-Swap3s83hP4hfygF_4noG

#_NotebookApp.iopub_data_rate_limit = 1000000.0 * 100 #100x the default

!pip install -q tqdm
import tqdm
from google.colab import auth
from googleapiclient.discovery import build
import io
from googleapiclient.http import MediaIoBaseDownload
from googleapiclient.http import MediaFileUpload


def download_file_from_google_drive(file_id = None, out_file_name = None):
  assert file_id is not None and out_file_name is not None
  auth.authenticate_user()
  drive_service = build('drive', 'v3')
  request = drive_service.files().get_media(fileId=file_id)
  downloaded = io.BytesIO()
  downloader = MediaIoBaseDownload(downloaded, request)
  done = False
  pbar = tqdm.tqdm(total=100, desc=out_file_name)

  while done is False:
    # _ is a placeholder for a progress object that we ignore.
    # (Our file is small, so we skip reporting progress.)
    status, done = downloader.next_chunk()
    pbar.update(status.progress() * 100)
    #print("Downloaded: ", int(status.progress() * 100))

  downloaded.seek(0)
  #print('Downloaded file contents are: {}'.format(downloaded.read()[:10]))
  with open(out_file_name, 'wb') as out:
    out.write(downloaded.read())
  print("Data downloaded to: ", out_file_name)
  return out_file_name
 

def save_file_to_google_drive(local_filename, dest_filename, mimetype = 'application/octet-stream'):
  auth.authenticate_user()
  drive_service = build('drive', 'v3')

  file_metadata = {
    'name': dest_filename,
    'mimeType': mimetype
  }
  media = MediaFileUpload(local_filename, 
                          mimetype=mimetype,
                          resumable=True)
  created = drive_service.files().create(body=file_metadata,
                                         media_body=media,
                                         fields='id').execute()
  print('File ID: {}'.format(created.get('id')))
  return created.get('id')

In [54]:
download_file_from_google_drive('1csHM7IpU3V-Swap3s83hP4hfygF_4noG', 'train.csv.zip')
download_file_from_google_drive('1o8Abgh9QBG30WakYhD7k13qubm4iqytg', 'test.csv.zip')

train.csv.zip: 2715.758477741821it [00:44, 97.75it/s]
test.csv.zip:   0%|          | 0/100 [00:00<?, ?it/s]

Data downloaded to:  train.csv.zip



test.csv.zip:   2%|▏         | 2.133228206336118/100 [00:02<01:54,  1.17s/it]
Exception in thread Thread-6:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/local/lib/python3.6/dist-packages/tqdm/_tqdm.py", line 148, in run
    for instance in self.tqdm_cls._instances:
  File "/usr/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

test.csv.zip: 2406.0196910493432it [00:44, 132.29it/s]

Data downloaded to:  test.csv.zip


In [0]:
import pandas as pd

In [0]:
train = pd.read_csv('train.csv.zip', compression = 'zip')
test = pd.read_csv('test.csv.zip', compression = 'zip')


In [0]:
maxlen = 500 #should analyze the distribution of sequences to determine right value
max_features = 75000

X_train = train["comment_text"].fillna("fillna").values
y_train = train[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values
X_test = test["comment_text"].fillna("fillna").values

tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X_train))
X_train = tokenizer.texts_to_sequences(X_train)
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)

In [58]:
print(len(X_train))
print("Max Length of sequence", max([len(seq) for seq in X_train]))
print("Max Index", max([max(seq) for seq in X_train]))

159571
Max Length of sequence 500
Max Index 74999


In [0]:
class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            local_fname = "test_model.h5"
            y_pred = self.model.predict(self.X_val, verbose=1)
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC-AUC - epoch: %d - score: %.6f \n" % (epoch+1, score))
            val_loss = logs.get("val_loss", "null")
            val_acc = logs.get("val_acc", "null")
            fname = "model-roc-{roc}-val_loss-{val_loss}-val_acc-{val_acc}-epoch-{epoch}.h5".format(roc=score, val_loss = val_loss, val_acc = val_acc, epoch=epoch)
            print("\n Saving model to local directory as:", local_fname)
            self.model.save(local_fname)
            print("\n save to local complete")
            print("\n starting upload to google drive", fname)
            fid = save_file_to_google_drive(local_fname, fname)
            print("\n upload completed: ", fid)
            
            
    def on_batch_end(self, epoch, logs={}):
        if epoch % 200 == 0:
            print('\n Starting ROC Eval on rand 1000 sample')
            sample = np.random.randint(X_val.shape[0], size = 1000)
            y_pred = self.model.predict(self.X_val[sample], verbose=1)
            y_val = self.y_val[sample]
            score = roc_auc_score(y_val, y_pred)
            print("\n ROC-AUC - epoch: %d - score: %.6f \n" % (epoch+1, score))   

In [0]:
embed_size = 100
def get_model():
    inp = Input(shape=(maxlen, ))
    x = Embedding(num_features, embed_size)(inp)
    x = SpatialDropout1D(0.2)(x)
    x = Bidirectional(GRU(80, return_sequences=True))(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    conc = concatenate([avg_pool, max_pool])
    outp = Dense(6, activation="sigmoid")(conc)
    
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model


In [84]:
model = get_model()


batch_size = 64
epochs = 6

X_tra, X_val, y_tra, y_val = train_test_split(X_train, y_train, train_size=0.95, random_state=233)
RocAuc = RocAucEvaluation(validation_data=(X_val, y_val), interval=1)



/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [85]:
model.load_weights("model.h5")

hist = model.fit(X_tra, y_tra, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val),
                 callbacks=[RocAuc])



Train on 151592 samples, validate on 7979 samples
Epoch 1/6
    64/151592 [..............................] - ETA: 3:18:11 - loss: 0.0442 - acc: 0.9844
 Starting ROC Eval on rand 1000 sample
1000/1000 [==============================] - 10s 10ms/step

 ROC-AUC - epoch: 1 - score: 0.981238 



/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:120: UserWarning: Method on_batch_end() is slow compared to the batch update (10.144026). Check your callbacks.
  % delta_t_median)


   128/151592 [..............................] - ETA: 5:26:21 - loss: 0.0412 - acc: 0.9870

/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:120: UserWarning: Method on_batch_end() is slow compared to the batch update (5.073380). Check your callbacks.
  % delta_t_median)


 12864/151592 [=>............................] - ETA: 51:35 - loss: 0.0378 - acc: 0.9857
 Starting ROC Eval on rand 1000 sample
1000/1000 [==============================] - 9s 9ms/step

 ROC-AUC - epoch: 201 - score: 0.985850 

 25664/151592 [====>.........................] - ETA: 46:18 - loss: 0.0381 - acc: 0.9855
 Starting ROC Eval on rand 1000 sample
 960/1000 [===========================>..] - ETA: 0s

...

1000/1000 [==============================] - 9s 9ms/step

 ROC-AUC - epoch: 1801 - score: 0.987410 

128064/151592 [========================>.....] - ETA: 8:37 - loss: 0.0405 - acc: 0.9845
 Starting ROC Eval on rand 1000 sample
1000/1000 [==============================] - 9s 9ms/step

 ROC-AUC - epoch: 2001 - score: 0.989228 

136128/151592 [=========================>....] - ETA: 5:40 - loss: 0.0405 - acc: 0.9845

140864/151592 [==========================>...] - ETA: 3:56 - loss: 0.0404 - acc: 0.9845
 Starting ROC Eval on rand 1000 sample
1000/1000 [==============================] - 9s 9ms/step

 ROC-AUC - epoch: 2201 - score: 0.983656 

2656/7979 [========>.....................] - ETA: 47s

7979/7979 [==============================] - 72s 9ms/step

 ROC-AUC - epoch: 1 - score: 0.985954 


 Saving model to local directory as: test_model.h5

 save to local complete

 starting upload to google drive
File ID: 1-3hIYWm1vC6An3mx7pJHUrdVfLkXX3g9

 upload completed:  1-3hIYWm1vC6An3mx7pJHUrdVfLkXX3g9
Epoch 2/6
    64/151592 [..............................] - ETA: 48:42 - loss: 0.0445 - acc: 0.9818
 Starting ROC Eval on rand 1000 sample
1000/1000 [==============================] - 9s 9ms/step

 ROC-AUC - epoch: 1 - score: 0.988937 



/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:120: UserWarning: Method on_batch_end() is slow compared to the batch update (9.190848). Check your callbacks.
  % delta_t_median)


KeyboardInterrupt: ignored

In [68]:

#'model-roc-0.978691-val_loss-0.0488-val_acc-0.9815.h5'
# Upload the file to Drive. See:
#
# https://developers.google.com/drive/v3/reference/files/create
# https://developers.google.com/drive/v3/web/manage-uploads


File ID: 1o4snyfur9XHRCh-H51oxC-PGRNwJ4SVU


In [87]:
X_test = test["comment_text"].fillna("fillna").values

X_test = tokenizer.texts_to_sequences(X_test)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)

y_pred = model.predict(X_test, batch_size=1024)


NameError: ignored

In [0]:
#y_pred.shape
labels = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
for idx, label in enumerate(labels):
  test[label] = y_pred[:, idx]
test.head()
headers = ["id"]+labels
test[headers].to_csv('submission.csv', index=False)

In [97]:
save_file_to_google_drive("submission.csv", "030518-lab-1-submission.csv", mimetype="text/csv")

File ID: 1Ru1HHYGXNVvIb3Rm-mnU8FIIx1hxW7R5


'1Ru1HHYGXNVvIb3Rm-mnU8FIIx1hxW7R5'